# Local Flood Mapping

In the context of this notebook we referred to "local flood mapping" as performing the computation on you own machine, as opposed to remote processing which will be explained in the next notebook.

We will begin by loading the `flood` module from the `dask_flood_mapper`. You could use the default Dask settings, but it is often good to fine tune te settings to you machine's specifications. We can do this by setting the Dask scheduler through the `Client` of `dask.distributed`.

In [ ]:
from dask_flood_mapper import flood
from dask.distributed import Client

Here we set the Dask Scheduler with the `Client`, where we avoid inter-worker communication which is common for working with `numpy` and `xarray` in this case.

In [ ]:
client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="12GB")
client

No we are ready to map a flood. As an example we use here storm Babet which hit the Danish and Northern coast of Germany at the 20<sup>th</sup> of October 2023 [Wikipedia](https://en.wikipedia.org/wiki/Storm_Babet). We target an area around Zingst at the Baltic coast of Northern Germany.

In [ ]:
time_range = "2022-10-11/2022-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
fd = flood.decision(bbox=bbox, datetime=time_range).compute()
fd

Since `flood.decision` does not make a distinction between Sentinel-1 observations over land or over sea, we need to mask pixels over water. For the example here we will load a mask distributed along with this package. But in general this step is left to the user's own discretion.

In [ ]:
from importlib.resources import files
import xarray as xr

data_text = files("dask_flood_mapper.data").joinpath("wcover.tif")
wcover = xr.open_dataarray(data_text)

Now we are ready to view our results. We use here `hvplot` to create an animation of the flood's extent over time.

In [ ]:
from bokeh.models import FixedTicker
import hvplot.xarray

fd = fd.where(wcover != 80)
colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
fd.hvplot.quadmesh(
    x="x",
    y="y",
    rasterize=True,
    geo=True,
    cmap=["rgba(0, 0, 1, 0.1)", "darkred"],
).opts(frame_height=400, colorbar_opts={**colorbar_opts})

In [ ]:
client.close()